In [130]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

year = '1972'  # Update the year accordingly

# URL of the website
url = f'https://rsssfbrasil.com/tablesae/br{year}.htm'

# Make a request to the website
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Initialize an empty list to store individual match results
    match_results = []

    # Use a regular expression to find match results
    pattern = r'\[([a-zA-Z]+\s+\d+(?:/\d{2})?)\]\s+((?:.*\n)+?)(?=\[|$)'
    matches = re.findall(pattern, soup.get_text())

    # Iterate through matches
    for match in matches:
        # Extract date and format it as [MMM dd]
        raw_date, match_text = match
        formatted_date = f'[{raw_date}]'

        # Split the match into lines
        lines = match_text.strip().split('\n')

        # Process each line (excluding the line with the date)
        for line in lines:
            # Use a regex to extract lines that match the specified pattern
            line_pattern = r'([a-zA-Z][\w\sàáâãäéèêëíìîïóòôõöúùûüç-]+)(\d+-\d+)\s*([\w\sàáâãäéèêëíìîïóòôõöúùûüç-]+)'
            line_match = re.match(line_pattern, line.strip(), re.UNICODE)
            
            if line_match:
                team1, score, team2 = line_match.groups()
                match_results.append({
                    'Data': formatted_date,
                    'Text': f'{team1} {score} {team2}'
                })

    # Create a DataFrame from the list of match results
    df = pd.DataFrame(match_results)

    # Save DataFrame to CSV
    df.to_csv('match_results.csv', index=False)
    print("DataFrame saved to 'match_results.csv'")
else:
    print(f"Error: {response.status_code}")

    


DataFrame saved to 'match_results.csv'


In [131]:
df


,Data,Text
0,[Sep 9],Botafogo 1-1 Santos
1,[Sep 9],Vitória 0-0 Remo
2,[Sep 9],Cruzeiro 0-0 América-MG
3,[Sep 10],Sergipe 1-3 Ceará
4,[Sep 10],ABC 0-0 CRB
...,...,...
347,[Dec 18],Botafogo 2-1 Santos
348,[Dec 18],Santa Cruz 2-1 Grêmio
349,[Dec 20],Palmeiras 1-1 Internacional
350,[Dec 20],Botafogo 2-1 Corinthians


In [116]:
# Assuming your DataFrame is named df
#new_row_index = 219  # Replace with the desired index for the new row
#existing_row_index = 0  # Replace with the index of the row you want to use as a reference

# Extract the date and MatchInfo from an existing row
#existing_date = df.loc[existing_row_index, 'Data']
#existing_match_info = df.loc[existing_row_index, 'MatchInfo']

# Concatenate the information for the new row
##new_row_data = {'Data': f"{existing_date} {existing_match_info}", 'MatchInfo': 'None'}

# Use loc to add the new row
##df.loc[new_row_index] = new_row_data
# Specify the indices to drop
indices_to_drop = (655)
#indices_to_drop.extend(range(471, 510))

df = df.drop(indices_to_drop)

# Reset the index after dropping rows
#df = df.reset_index(drop=True)

# Drop the 'MatchInfo' column
#df = df.drop(['Text'], axis=1)

# Extract information using a regular expression
df[['Casa', 'GC', 'GF', 'Fora']] = df['Text'].str.extract(
    r'([\w\sàáâãäéèêëíìîïóòôõöúùûüç-]+)(\d+)-(\d+)\s+([\w\sàáâãäéèêëíìîïóòôõöúùûüç-]+)')

df = df.drop(['Text'], axis=1)

# Display the updated DataFrame
df

,Data,Casa,GC,GF,Fora
0,[Aug 25],Corinthians,0,0,América-GB
1,[Aug 25],CEUB,0,0,Botafogo
2,[Aug 25],Paysandu,2,1,Internacional
3,[Aug 25],Tiradentes,0,0,Fluminense
4,[Aug 25],Figueirense,0,0,Coritiba
...,...,...,...,...,...
651,[Feb 13/74],Cruzeiro,0,1,Palmeiras
652,[Feb 17/74],Palmeiras,2,1,Internacional
653,[Feb 17/74],Cruzeiro,1,0,São Paulo
654,[Feb 20/74],Internacional,1,0,Cruzeiro


In [117]:
unique_values = df['Casa'].unique()
unique_values_as_strings = [str(value) for value in unique_values]
unique_values_sorted = sorted(unique_values_as_strings)
print(unique_values_sorted)

# Remove leading and trailing spaces
team_names_stripped = [name.strip() for name in unique_values_sorted]

# Print the result
print(team_names_stripped)



['América-GB      ', 'América-MG      ', 'América-RN      ', 'Atlético-MG     ', 'Atlético-PR     ', 'Bahia           ', 'Botafogo        ', 'CEUB            ', 'CRB             ', 'Ceará           ', 'Comercial       ', 'Corinthians     ', 'Coritiba        ', 'Cruzeiro        ', 'Desportiva      ', 'Figueirense     ', 'Flamengo        ', 'Fluminense      ', 'Fortaleza       ', 'Goiás           ', 'Grêmio          ', 'Guarani         ', 'Internacional   ', 'Moto Clube      ', 'Nacional        ', 'Náutico         ', 'Olaria          ', 'Palmeiras  ', 'Palmeiras       ', 'Paysandu        ', 'Portuguesa      ', 'Remo            ', 'Rio Negro       ', 'Santa Cruz      ', 'Santos          ', 'Sergipe         ', 'Sport           ', 'São Paulo       ', 'Tiradentes      ', 'Vasco da Gama   ', 'Vitória         ']
['América-GB', 'América-MG', 'América-RN', 'Atlético-MG', 'Atlético-PR', 'Bahia', 'Botafogo', 'CEUB', 'CRB', 'Ceará', 'Comercial', 'Corinthians', 'Coritiba', 'Cruzeiro', 'Desportiva', 

In [118]:
# Assuming 'Casa' is the column you want to replace
df['Casa'] = df['Casa'].replace(unique_values_sorted, team_names_stripped)

df

,Data,Casa,GC,GF,Fora
0,[Aug 25],Corinthians,0,0,América-GB
1,[Aug 25],CEUB,0,0,Botafogo
2,[Aug 25],Paysandu,2,1,Internacional
3,[Aug 25],Tiradentes,0,0,Fluminense
4,[Aug 25],Figueirense,0,0,Coritiba
...,...,...,...,...,...
651,[Feb 13/74],Cruzeiro,0,1,Palmeiras
652,[Feb 17/74],Palmeiras,2,1,Internacional
653,[Feb 17/74],Cruzeiro,1,0,São Paulo
654,[Feb 20/74],Internacional,1,0,Cruzeiro


In [119]:
unique_values = df['Casa'].unique()
unique_values_as_strings = [str(value) for value in unique_values]
unique_values_sorted = sorted(unique_values_as_strings)
print(unique_values_sorted)

['América-GB', 'América-MG', 'América-RN', 'Atlético-MG', 'Atlético-PR', 'Bahia', 'Botafogo', 'CEUB', 'CRB', 'Ceará', 'Comercial', 'Corinthians', 'Coritiba', 'Cruzeiro', 'Desportiva', 'Figueirense', 'Flamengo', 'Fluminense', 'Fortaleza', 'Goiás', 'Grêmio', 'Guarani', 'Internacional', 'Moto Clube', 'Nacional', 'Náutico', 'Olaria', 'Palmeiras', 'Paysandu', 'Portuguesa', 'Remo', 'Rio Negro', 'Santa Cruz', 'Santos', 'Sergipe', 'Sport', 'São Paulo', 'Tiradentes', 'Vasco da Gama', 'Vitória']


In [120]:



df.replace({'Atlético-PR': 'Athletico-PR',
            'América-GB': 'America-RJ',
            'Botafogo-RJ': 'Botafogo',
            'Corinthians      ': 'Corinthians',
            'Comercial': 'Comercial-MS',
            'Desportiva': 'Desportiva Capixaba',
            'Flamengo-RJ': 'Flamengo',
            'Fluminense-RJ': 'Fluminense',
            'Mixto': 'Mixto-MT',
            'Nacional': 'Nacional-AM',
            'Operário': 'Operário-MS',
            'Rio Branco': 'Rio Branco-ES',
            'Rio Negro': 'Rio Negro-AM',
            'Tiradentes': 'Tiradentes-PI',
            'Vasco da Gama': 'Vasco',             
            }, inplace=True)

df

,Data,Casa,GC,GF,Fora
0,[Aug 25],Corinthians,0,0,America-RJ
1,[Aug 25],CEUB,0,0,Botafogo
2,[Aug 25],Paysandu,2,1,Internacional
3,[Aug 25],Tiradentes-PI,0,0,Fluminense
4,[Aug 25],Figueirense,0,0,Coritiba
...,...,...,...,...,...
651,[Feb 13/74],Cruzeiro,0,1,Palmeiras
652,[Feb 17/74],Palmeiras,2,1,Internacional
653,[Feb 17/74],Cruzeiro,1,0,São Paulo
654,[Feb 20/74],Internacional,1,0,Cruzeiro


In [121]:
import re

# Define your regular expression
date_pattern = r'\[([a-zA-Z]+ \d{1,2}/\d{2})\]'

# Create a filter using str.contains and the regular expression
filt = df['Data'].str.contains(date_pattern, regex=True)

# Apply the filter to the DataFrame
filtered_df = df[filt]
df = df[~filt]

# Display the filtered DataFrame
print(filtered_df)
print(df)


            Data           Casa GC GF           Fora
560  [Jan 12/74]    Corinthians  2  2          Bahia
561  [Jan 13/74]      Palmeiras  0  0       Coritiba
562  [Jan 13/74]    Atlético-MG  0  0  Internacional
563  [Jan 13/74]          Vasco  1  2     América-MG
564  [Jan 13/74]          Ceará  0  2  Tiradentes-PI
..           ...            ... .. ..            ...
651  [Feb 13/74]       Cruzeiro  0  1      Palmeiras
652  [Feb 17/74]      Palmeiras  2  1  Internacional
653  [Feb 17/74]       Cruzeiro  1  0      São Paulo
654  [Feb 20/74]  Internacional  1  0       Cruzeiro
656  [Feb 20/74]      Palmeiras  0  0      São Paulo

[96 rows x 5 columns]
         Data           Casa GC GF                 Fora
0    [Aug 25]    Corinthians  0  0           America-RJ
1    [Aug 25]           CEUB  0  0             Botafogo
2    [Aug 25]       Paysandu  2  1        Internacional
3    [Aug 25]  Tiradentes-PI  0  0           Fluminense
4    [Aug 25]    Figueirense  0  0             Coritiba
..   

C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_10700\2429067132.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filt = df['Data'].str.contains(date_pattern, regex=True)


In [122]:
df['Data'] = df['Data'].str.replace('[', '').str.replace(']', '')
month_mapping = {
    'Jan': '01',
    'Feb': '02',
    'Mar': '03',
    'Apr': '04',
    'May': '05',
    'Jun': '06',
    'Jul': '07',
    'Aug': '08',
    'Sep': '09',
    'Oct': '10',
    'Nov': '11',
    'Dec': '12',    
}



# Assuming df is your DataFrame with the 'Data' column in datetime64[ns] format
df['Data'] = df['Data'].astype(str)
df['Data'] = df['Data'].replace(month_mapping, regex=True)
df['Data'] = df['Data'].str.replace(' ', '-')  # Remove brackets from the date string
df['Data'] = pd.to_datetime(df['Data'], format='%m-%d')  # Convert to datetime format
df['Data'] = df['Data'].apply(lambda x: x.replace(year=int(year))).dt.strftime('%Y-%m-%d')

# Now df['Data'] should have the correct date with the specified year


C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_10700\1113887071.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Data'] = df['Data'].str.replace('[', '').str.replace(']', '')
C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_10700\1113887071.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Data'] = df['Data'].astype(str)
C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_10700\1113887071.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [123]:
df2 = df
df = filtered_df

In [124]:
import pandas as pd

# Extract date part without brackets
df['Data'] = df['Data'].str.replace('[', '').str.replace(']', '')

# Mapping for month abbreviations
month_mapping = {
    'Jan': '01',
    'Feb': '02',
    'Mar': '03',
    'Apr': '04',
    'May': '05',
    'Jun': '06',
    'Jul': '07',
    'Aug': '08',
    'Sep': '09',
    'Oct': '10',
    'Nov': '11',
    'Dec': '12',    
}

# Replace month abbreviations with numeric values
df['Data'] = df['Data'].replace(month_mapping, regex=True)

# Replace spaces and slashes in the date string
df['Data'] = df['Data'].str.replace(' ', '-').str.replace('/', '-')

# Convert to datetime format
df['Data'] = pd.to_datetime(df['Data'], format='%m-%d-%y')

# Format the date as YYYY-MM-DD
df['Data'] = df['Data'].dt.strftime('%Y-%m-%d')

print(df)


           Data           Casa GC GF           Fora
560  1974-01-12    Corinthians  2  2          Bahia
561  1974-01-13      Palmeiras  0  0       Coritiba
562  1974-01-13    Atlético-MG  0  0  Internacional
563  1974-01-13          Vasco  1  2     América-MG
564  1974-01-13          Ceará  0  2  Tiradentes-PI
..          ...            ... .. ..            ...
651  1974-02-13       Cruzeiro  0  1      Palmeiras
652  1974-02-17      Palmeiras  2  1  Internacional
653  1974-02-17       Cruzeiro  1  0      São Paulo
654  1974-02-20  Internacional  1  0       Cruzeiro
656  1974-02-20      Palmeiras  0  0      São Paulo

[96 rows x 5 columns]


C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_10700\2646166599.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Data'] = df['Data'].str.replace('[', '').str.replace(']', '')
C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_10700\2646166599.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Data'] = df['Data'].replace(month_mapping, regex=True)
C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_10700\2646166599.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [125]:
frames = [df, df2]
df3 = pd.concat(frames)
df = df3

In [126]:
#Step 8: cleaning data frame
#creates a new column with the for the competition name

#creates new columns with competition name and season
df = df.assign(Comp='Série A')
df = df.assign(Temp=f'{year}')

#comparing scores to display the results in the second dataframe
df.loc[df['GC'] == df['GF'], 'Res'] = 'E'
df.loc[df['GC'] > df['GF'], 'Res'] = 'VC' 
df.loc[df['GC'] < df['GF'], 'Res'] = 'VV'
# create a new column to store the season information in the second data frame

#organizes de columns of the sacond dataframe
new_order = ['Comp','Temp','Data','Casa','GC', 'GF','Fora','Res']
df= df.reindex(columns=new_order)
df


#drops unvalid data
df.dropna(axis=0, how='all', inplace=True)

df = df.sort_values(by='Data')


#atributes a pontuation to each game
df['PC'] = df.apply(lambda x: 3 if x['Res'] == 'VC' else
                              1 if x['Res'] == 'E' else 0, axis=1)
df['PF'] = df.apply(lambda x: 3 if x['Res'] == 'VV' else
                              1 if x['Res'] == 'E' else 0, axis=1)

#atributes a number of games for each season
df['J'] = df.groupby('Temp').cumcount() + 1
#corrects any possible issue before saving as new file
df

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
8,Série A,1973,1973-08-25,Sport,0,0,Fortaleza,E,1,1,1
13,Série A,1973,1973-08-25,Ceará,2,2,Náutico,E,1,1,2
12,Série A,1973,1973-08-25,Comercial-MS,0,1,Flamengo,VV,0,3,3
11,Série A,1973,1973-08-25,Goiás,0,0,Olaria,E,1,1,4
10,Série A,1973,1973-08-25,Desportiva Capixaba,2,0,Sergipe,VC,3,0,5
...,...,...,...,...,...,...,...,...,...,...,...
651,Série A,1973,1974-02-13,Cruzeiro,0,1,Palmeiras,VV,0,3,652
652,Série A,1973,1974-02-17,Palmeiras,2,1,Internacional,VC,3,0,653
653,Série A,1973,1974-02-17,Cruzeiro,1,0,São Paulo,VC,3,0,654
656,Série A,1973,1974-02-20,Palmeiras,0,0,São Paulo,E,1,1,655


In [127]:
df.to_csv(f'campeonato_brasileiro_{year}.csv', index=False)


In [128]:
# Merge dataframes and sort by date
df2 = pd.read_csv('campeonato_brasileiro_71_76.csv')
frames = [df, df2]
df3 = pd.concat(frames)
df3["Data"] = pd.to_datetime(df3["Data"], errors='coerce')  # Convert "Data" to datetime format
df3 = df3.drop_duplicates()
df3 = df3.sort_values(by='Data')
df3.to_csv('campeonato_brasileiro_71-76.csv', index=False)


In [129]:
df3

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
8,Série A,1973,1973-08-25,Sport,0,0,Fortaleza,E,1,1,1
15,Série A,1973,1973-08-25,América-RN,0,0,Rio Negro-AM,E,1,1,17
6,Série A,1973,1973-08-25,Bahia,1,1,São Paulo,E,1,1,16
0,Série A,1973,1973-08-25,Corinthians,0,0,America-RJ,E,1,1,15
1,Série A,1973,1973-08-25,CEUB,0,0,Botafogo,E,1,1,14
...,...,...,...,...,...,...,...,...,...,...,...
405,Série A,1976,1976-11-28,Fluminense,1,0,Bahia,VC,3,0,406
406,Série A,1976,1976-11-28,Atlético-MG,0,0,Grêmio,E,1,1,407
408,Série A,1976,1976-12-05,Internacional,2,1,Atlético-MG,VC,3,0,409
407,Série A,1976,1976-12-05,Fluminense,1,1,Corinthians,E,1,1,408


In [5]:
unique_values = df2['fora'].unique()
unique_values_as_strings = [str(value) for value in unique_values]
unique_values_sorted = sorted(unique_values_as_strings)
print(unique_values_sorted)

NameError: name 'df2' is not defined

In [212]:
filt = ((df3['Temp'] == '1957'))
df3[filt]

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
1,Paulista,1957,1957-06-09,Portuguesa Santista,3,1,Ferroviária (SP),VC,3,0,1
7,Paulista,1957,1957-06-09,XV de Jaú,4,1,Ypiranga (SP),VC,3,0,2
2,Paulista,1957,1957-06-09,XV de Piracicaba,1,0,Taubaté,VC,3,0,8
4,Paulista,1957,1957-06-09,Portuguesa,2,2,Ponte Preta,E,1,1,7
3,Paulista,1957,1957-06-09,Nacional (SP),4,1,Linense,VC,3,0,6
...,...,...,...,...,...,...,...,...,...,...,...
279,Paulista,1957,1958-01-26,Linense,2,2,Ypiranga (SP),E,1,1,367
278,Paulista,1957,1958-01-26,Noroeste,3,2,Guarani,VC,3,0,364
276,Paulista,1957,1958-01-26,Nacional (SP),2,0,São Bento (SP),VC,3,0,365
277,Paulista,1957,1958-01-26,Ferroviária (SP),3,1,Taubaté,VC,3,0,366


In [196]:
df3 = df3[df3['Temp'] != '1957']

In [218]:
unique_values = df3['Temp'].unique()
unique_values_as_strings = [str(value) for value in unique_values]
unique_values_sorted = sorted(unique_values_as_strings)
print(unique_values_sorted)

['1955', '1956', '1957', '1957', '1958', '1959', '1960', '1961', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1973', '1974', '1975']


In [ ]:
# Assuming your DataFrame is named df
new_row_index = 372  # Replace with the desired index for the new row
existing_row_index = 0  # Replace with the index of the row you want to use as a reference

# Extract the date and MatchInfo from an existing row
existing_date = df.loc[existing_row_index, 'Data']
existing_match_info = df.loc[existing_row_index, 'MatchInfo']

# Concatenate the information for the new row
new_row_data = {'Data': f"{existing_date} {existing_match_info}", 'MatchInfo': 'None'}

# Use loc to add the new row
df.loc[new_row_index] = new_row_data
